# Analysis of in silico samples

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime


In [ ]:
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
# VARS
dir_diversity_output = '../results_diversity'
dir_reads_fastq = '../data/artificial_reads/'
fastq_basename = 'POOL_R1.fastq.gz'
dir_results_profiling = '../results_profiling/artificial_reads'
pools_file="../data/artificial_reads/samples_profiling.txt"

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')
os.makedirs(f'{dir_diversity_output}/{today}', exist_ok=True)

In [ ]:
# GENERAL VARIABLES
POOL_list = !cat {pools_file}
profilers = ['kaiju', 'kraken_2', 'krakenuniq', 'centrifuge']

In [ ]:
# MINOR PROCESSING FUNCTIONS
def process_df(df):
    # Reverses the philogenetic order of the taxa, and removes "root" and "cellular organisms" labels
    lineage_vals = df.lineage.values
    new_lineage_vals = []
    for lineage in lineage_vals:
        val = ';'.join(lineage.split(';')[::-1]).replace('root;', '').replace('cellular organisms;', '')
        new_lineage_vals.append(val)
    df.lineage = new_lineage_vals
    return df

def get_FASTQ_len(POOL_list):
    # Obtains a list of the size of the FASTQs before profiling.
    dict_FASTQ_len = {}
    
    for POOL in POOL_list:
        n_counts_fastq = !gzip -dc {dir_reads_fastq}/{fastq_basename.replace('POOL', POOL)} | wc -l
        dict_FASTQ_len[POOL] = int(n_counts_fastq[0])
    
    return dict_FASTQ_len


In [ ]:
# MAJOR PROCESSING FUNCTIONS

def create_POOL_table(POOL, profilers, dict_FASTQ_len, cutoff_NA, cutoff_CV, cutoff_min_reads):
    index_df_POOL = [] # index of taxonomy members to create a merged table with all methods
    list_df_pools = []

    # Loading the tables. We (1) reverse the phylogenetic lineage to start by root and and by species, (2) sort index by taxonomy and (3) rename count column to the method
    for tax_method in profilers:
        df_POOL_method = process_df(pd.read_csv(f'{dir_results_profiling}/{tax_method}/{POOL}.report.standardised', sep='\t', index_col='taxonomy_id')).rename(columns={'count': tax_method})
        list_df_pools.append(df_POOL_method)
        index_df_POOL += df_POOL_method.index.tolist()
    
    index_df_POOL = list(set(index_df_POOL))
    
    # Creating the table and merging count columns
    df_POOL = pd.DataFrame(index=index_df_POOL, columns= ['name', 'lineage'] + profilers)

    for df_POOL_x, name_x in zip(list_df_pools, profilers):
        df_POOL.loc[df_POOL_x.index, ['name', 'lineage', name_x]] = df_POOL_x.loc[df_POOL_x.index, ['name', 'lineage', name_x]]

    # Apply normalisation based on other pools - this step takes the number of reads of the FASTQs and corrects the counts by dividing it by the mean number of counts, so that a a FASTQ with more total reads has fewer normalised counts
    cols_POOL_norm = [f'{i}_norm' for i in profilers]
    mean_FASTQ_len = np.median(np.array(list(dict_FASTQ_len.values())))
    correction_factor = mean_FASTQ_len / dict_FASTQ_len[POOL]

    for col_POOL in profilers:
        df_POOL[f'{col_POOL}_norm'] = df_POOL[col_POOL] * correction_factor

    # Calculate simple stats
    df_POOL.loc[:, 'raw_mean'] = np.mean(df_POOL.loc[:, profilers], axis=1).astype(float)

    df_POOL.loc[:, 'mean'] = np.mean(df_POOL.loc[:, cols_POOL_norm], axis=1).astype(float)
    df_POOL.loc[:, 'std'] = np.std(df_POOL.loc[:, cols_POOL_norm], axis=1)


    df_POOL = df_POOL[df_POOL['mean'] > 0]  # Filter step because in some cases it was 0 and CV y¡would yield NAN

    df_POOL.loc[:, 'CV'] = df_POOL.loc[:, 'std'] / df_POOL.loc[:, 'mean']


    # Calculate the number of reads in the fastq to obtain the relative abundance of the counts
    df_POOL.loc[:, 'mean (%)'] = 100 * df_POOL.loc[:, 'mean'] / mean_FASTQ_len


    df_POOL = df_POOL.sort_values(by='mean', ascending=False)


    
    # We use some quality metrics to flag and remove "bad quality" samples:
    #   cutoff_CV to flag species that have very variable counts across profilers
    #   cutoff_min_reads and cutoff_min_sum_reads to flag species that have a low count in one and in all profiling counts.
    #   cutoff_NA to remove species that are only present in 2 or fewer samples
    cutoff_median_reads = int(cutoff_min_reads * 2.5)
    df_POOL[['quality_CV', 'quality_min_reads', 'quality_sum_reads', 'quality_NA']] = 0

    df_POOL.loc[df_POOL.loc[:, 'CV'] > cutoff_CV, 'quality_CV'] += 1
    df_POOL.loc[df_POOL.loc[:, cols_POOL_norm].min(1, skipna=True) < cutoff_min_reads, 'quality_min_reads'] +=  1
    df_POOL.loc[df_POOL.loc[:, cols_POOL_norm].median(1, skipna=True) < cutoff_median_reads, 'quality_min_reads'] +=  1
    df_POOL.loc[df_POOL.loc[:, cols_POOL_norm].isna().sum(1) > cutoff_NA, 'quality_NA'] += 1
    df_POOL.loc[df_POOL.loc[:, cols_POOL_norm].isna().sum(1) == len(profilers) - 1, 'quality_NA'] += 1  # If only one profiler shows the information, we remove it
    df_POOL['quality'] = df_POOL.loc[:, ['quality_CV', 'quality_min_reads', 'quality_sum_reads', 'quality_NA']].sum(1)
    # df_POOL = df_POOL[~ np.isnan(df_POOL['CV'])]

    # We select species with 0 or 1 flag. We are restrictive in that sense to avoid flagging "low quality" species
    df_POOL_cutoff = df_POOL[df_POOL['quality'] < 2] 


    df_POOL.to_csv(f'../results_diversity/{today}/{POOL}.diversity_raw.tsv', sep='\t')
    df_POOL_cutoff .to_csv(f'../results_diversity/{today}/{POOL}.diversity_cutoff.tsv', sep='\t')

    return df_POOL, df_POOL_cutoff



In [ ]:
# PLOTTING FUNCTIONS

def plot_allPOOL_correlations(POOL_list, profilers, corr_method='spearman'):
    ncols = int(len(POOL_list) ** 0.5)
    nrows = int(len(POOL_list) // ncols) + int(len(POOL_list) % ncols != 0)

    

    list_mean_heatmaps = []

    for type_plot_idx, type_plot in enumerate(['raw', 'cutoff']):
        _, axs= plt.subplots(nrows, ncols, figsize=(4 * ncols, 4 * nrows))
        corr_mat_list = [] # this is to later sum all correlations and do a big plot
        for ax_int, POOL in enumerate(POOL_list):
            df_POOL = pd.read_csv(f'../results_diversity/{today}/{POOL}.diversity_{type_plot}.tsv', sep='\t')
            df_corr = np.log10(df_POOL.loc[:, profilers].astype(float) + 1)
            corr_mat = df_corr.corr(method=corr_method)
            corr_mat_list.append(corr_mat)

            sns.heatmap(corr_mat, cmap='Blues', annot=True, ax=axs.ravel()[ax_int])
            axs.ravel()[ax_int].set_title(POOL)
        
        plt.suptitle(f'Correlation ({corr_method}, {type_plot})')
        plt.tight_layout()

        plt.savefig(f'../results_diversity/{today}/correlation_{corr_method}_{type_plot}.png', dpi=300)

        # Create the mean heatmap

        mean_ht = corr_mat_list[0]
        NaNs_mat = np.isnan(mean_ht).astype(int)

        for corr_mat in corr_mat_list[1:]:
            mean_ht = np.nansum(np.dstack((mean_ht,corr_mat)), 2) # This function is used to sum avoiding NaNs
            NaNs_mat += np.isnan(corr_mat).astype(int) # To calculate the mean properly, we need to divided buy the number of non NaN elements.
        mean_ht /= (len(corr_mat_list) - NaNs_mat)

        list_mean_heatmaps.append(mean_ht)

    _, axs_all = plt.subplots(1, 2, figsize=(4 * 2, 4 * 1))

    for type_plot_idx, type_plot in enumerate(['raw', 'cutoff']):
        sns.heatmap(list_mean_heatmaps[type_plot_idx], cmap='Blues', annot=True, ax=axs_all.ravel()[type_plot_idx])
        axs_all.ravel()[type_plot_idx].set_title(type_plot)

    plt.tight_layout()
    plt.savefig(f'../results_diversity/{today}/correlation_{corr_method}_mean.png', dpi=300)

    


In [ ]:
dict_FASTQ_len = {'ARTIFICIAL': 200000128/4}

In [ ]:
dfpool, dfpoolcut = create_POOL_table('ARTIFICIAL', profilers, dict_FASTQ_len, cutoff_NA=1, cutoff_CV=1.0, cutoff_min_reads=100)

display(dfpool)


In [ ]:
display(dfpool.sum()[['kaiju', 'kraken_2', 'krakenuniq', 'centrifuge']])
display(dfpool[dfpool['name'] != 'Homo'].sum()[['kaiju', 'centrifuge', 'kraken_2', 'krakenuniq',]])

In [ ]:
dfpool

In [ ]:
list_taxids = [1912216, 37914, 1678, 1578, 816, 841, 40323, 13687, 29574, 683756, 5052, 55193, 4930, 118259, 
               5598, 2948645, 1984786, 11646, 2560188, 10375]
list_n_reads = [375000, 250000, 250000, 125000, 125000, 125000, 75000, 75000, 50000, 25000, 250000, 125000, 
                125000, 75000, 75000, 125000, 75000, 75000, 75000, 25000, ]

# Some species may not make the cut to dfpoolcut, so we select the index from the
dfpoolcut = dfpool.loc[list_taxids + [i for i in dfpoolcut.index if i not in list_taxids]]

dfpoolcut = dfpoolcut.fillna(0)
dfpoolcut.loc[:, 'reads'] = 0

dfpoolcut.loc[list_taxids, 'reads'] = list_n_reads


dfpoolcut = dfpoolcut[['name', 'reads', 'kaiju', 'kraken_2', 'krakenuniq', 'centrifuge', 'mean']]
dfpoolcut = dfpoolcut[dfpoolcut['name'] != 'Homo']
display(dfpoolcut)

len(dfpoolcut)




In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3.5, 3))

g = sns.swarmplot([np.log10(dfpoolcut[dfpoolcut['reads'] > 0]['mean'].values), np.log10(dfpoolcut[dfpoolcut['reads'] == 0]['mean'].values)],)
g.set_xticklabels(['Existing genera', 'Non-existing genera'])
g.set_ylabel("log$_{10}$(Mean number of reads \nacross profilers)")

display(dfpoolcut[dfpoolcut['reads'] > 0]['mean'].values.mean(), dfpoolcut[dfpoolcut['reads'] == 0]['mean'].values.mean())
plt.savefig(f"../results_diversity/{today}/mean_reads_artificial.png", dpi=300, bbox_inches='tight')

## Analysing reads assigned to existing species

In [ ]:
dfpoolcut_with_reads = dfpoolcut[dfpoolcut['reads'] > 0]

dfpoolcut_with_reads.loc[:, 'kaiju_norm'] = 1 + (dfpoolcut_with_reads['kaiju'] - dfpoolcut_with_reads['reads']) / dfpoolcut_with_reads['reads']
dfpoolcut_with_reads.loc[:, 'kraken_2_norm'] = 1 + (dfpoolcut_with_reads['kraken_2'] - dfpoolcut_with_reads['reads']) / dfpoolcut_with_reads['reads']
dfpoolcut_with_reads.loc[:, 'krakenuniq_norm'] = 1 + (dfpoolcut_with_reads['krakenuniq'] - dfpoolcut_with_reads['reads']) / dfpoolcut_with_reads['reads']
dfpoolcut_with_reads.loc[:, 'centrifuge_norm'] = 1 + (dfpoolcut_with_reads['centrifuge'] - dfpoolcut_with_reads['reads']) / dfpoolcut_with_reads['reads']
dfpoolcut_with_reads.loc[:, 'mean_norm'] = 1 + (dfpoolcut_with_reads['mean'] - dfpoolcut_with_reads['reads']) / dfpoolcut_with_reads['reads']

display(dfpoolcut_with_reads)

df_stats = pd.DataFrame({'MEDIAN': dfpoolcut_with_reads[['kaiju_norm', 'kraken_2_norm', 'krakenuniq_norm', 'centrifuge_norm', 'mean_norm']].median(), 
                         'CV_MAD': dfpoolcut_with_reads[['kaiju_norm', 'kraken_2_norm', 'krakenuniq_norm', 'centrifuge_norm', 'mean_norm']].mad() / dfpoolcut_with_reads[['kaiju_norm', 'kraken_2_norm', 'krakenuniq_norm', 'centrifuge_norm', 'mean_norm']].median(),
                         'MEAN': dfpoolcut_with_reads[['kaiju_norm', 'kraken_2_norm', 'krakenuniq_norm', 'centrifuge_norm', 'mean_norm']].mean(), 
                         'CV': dfpoolcut_with_reads[['kaiju_norm', 'kraken_2_norm', 'krakenuniq_norm', 'centrifuge_norm', 'mean_norm']].std() / dfpoolcut_with_reads[['kaiju_norm', 'kraken_2_norm', 'krakenuniq_norm', 'centrifuge_norm', 'mean_norm']].mean()})
display(df_stats)

In [ ]:
fig = plt.figure(figsize=(10,3))
plt.plot([1,1],[0,3.5], c='#bcbcbc')
bw_adjust = 0.25
sns.kdeplot(dfpoolcut_with_reads.loc[:, 'kaiju_norm'], bw_adjust=bw_adjust, label='Kaiju')
sns.kdeplot(dfpoolcut_with_reads.loc[:, 'kraken_2_norm'], bw_adjust=bw_adjust, label='Kraken2')
sns.kdeplot(dfpoolcut_with_reads.loc[:, 'krakenuniq_norm'], bw_adjust=bw_adjust, label='KrakenUniq')
sns.kdeplot(dfpoolcut_with_reads.loc[:, 'centrifuge_norm'], bw_adjust=bw_adjust, label='Centrifuge')
sns.kdeplot(dfpoolcut_with_reads.loc[:, 'mean_norm'], bw_adjust=bw_adjust, label='Mean')
plt.legend()

In [ ]:
fig = plt.figure(figsize=(7,2.5), dpi=300)
plt.plot([1, 1], [0.9, 5.1], c='#bcbcbc')
random_noise = np.random.normal(loc=0.0, scale=0.02, size=len(dfpoolcut_with_reads))

plt.scatter(dfpoolcut_with_reads.loc[:, 'kaiju_norm'] + random_noise, [1] * len(dfpoolcut_with_reads), label='Kaiju', alpha=0.8)
plt.plot([np.mean(dfpoolcut_with_reads.loc[:, 'kaiju_norm'])] * 2, [1 - 0.25, 1 + 0.25], alpha=0.8, c='#454545')

plt.scatter(dfpoolcut_with_reads.loc[:, 'kraken_2_norm'] + random_noise, [2] * len(dfpoolcut_with_reads), label='Kraken2', alpha=0.8)
plt.plot([np.mean(dfpoolcut_with_reads.loc[:, 'kraken_2_norm'])] * 2, [2 - 0.25, 2 + 0.25], alpha=0.8, c='#454545')

plt.scatter(dfpoolcut_with_reads.loc[:, 'krakenuniq_norm'] + random_noise, [3] * len(dfpoolcut_with_reads), label='KrakenUniq', alpha=0.8)
plt.plot([np.mean(dfpoolcut_with_reads.loc[:, 'krakenuniq_norm'])] * 2, [3 - 0.25, 3 + 0.25], alpha=0.8, c='#454545')

plt.scatter(dfpoolcut_with_reads.loc[:, 'centrifuge_norm'] + random_noise, [4] * len(dfpoolcut_with_reads), label='Centrifuge', alpha=0.8)
plt.plot([np.mean(dfpoolcut_with_reads.loc[:, 'centrifuge_norm'])] * 2, [4 - 0.25, 4 + 0.25], alpha=0.8, c='#454545')

plt.scatter(dfpoolcut_with_reads.loc[:, 'mean_norm'] + random_noise, [5] * len(dfpoolcut_with_reads), label='Mean')
plt.plot([np.mean(dfpoolcut_with_reads.loc[:, 'mean_norm'])] * 2, [5 - 0.25, 5 + 0.25], alpha=0.8, c='#454545')

plt.yticks([])
plt.legend()
plt.xlabel('Efficiency')

Each of the profilers outputs a different pattern:
- Kaiju and kraken2 seem to work similarly, assigning, in mean, half of the reads to the species. 
- Kraquenuniq: It shows the highest similarity to the original number of reads, except for especies that does not recognize, probably because they are not integrated in the database.
- Centrifuge: In general it detects reads in a much higher proportion (2-3 times) than the expected value, although some species (some of them "common" like Dietzia) fail to be assigned.

Ironically, despite the underrepresentation of reads in 2/4 profiles, the mean value approaches the expected value more.

## Analysing reads assigned to non-existing species

In [ ]:
dfpoolcut_non_reads = dfpoolcut[dfpoolcut['reads'] == 0]
dfpoolcut_non_reads.loc[:, 'kaiju_norm'] = np.log10(dfpoolcut_non_reads['kaiju'] + 1)
dfpoolcut_non_reads.loc[:, 'kraken_2_norm'] = np.log10(dfpoolcut_non_reads['kraken_2'] + 1)
dfpoolcut_non_reads.loc[:, 'krakenuniq_norm'] = np.log10(dfpoolcut_non_reads['krakenuniq'] + 1)
dfpoolcut_non_reads.loc[:, 'centrifuge_norm'] = np.log10(dfpoolcut_non_reads['centrifuge'] + 1)
dfpoolcut_non_reads.loc[:, 'mean_norm'] = np.log10(dfpoolcut_non_reads['mean'] + 1)

dfpoolcut_non_reads

In [ ]:
fig = plt.figure(figsize=(10,3))
plt.plot([0, 0],[0,3.5], c='#bcbcbc')
bw_adjust = 0.3
sns.kdeplot(dfpoolcut_non_reads.loc[:, 'kaiju_norm'], bw_adjust=bw_adjust, label='Kaiju')
sns.kdeplot(dfpoolcut_non_reads.loc[:, 'kraken_2_norm'], bw_adjust=bw_adjust, label='Kraken2')
sns.kdeplot(dfpoolcut_non_reads.loc[:, 'krakenuniq_norm'], bw_adjust=bw_adjust, label='KrakenUniq')
sns.kdeplot(dfpoolcut_non_reads.loc[:, 'centrifuge_norm'], bw_adjust=bw_adjust, label='Centrifuge')
sns.kdeplot(dfpoolcut_non_reads.loc[:, 'mean_norm'], bw_adjust=bw_adjust, label='Mean')
plt.legend()
plt.xticks([0, 1, 2, 3, 4], [0, 10, 100, 1000, 10000])

In [ ]:
fig = plt.figure(figsize=(7,2.5), dpi=300)
plt.plot([0, 0], [0.9, 5.1], c='#bcbcbc')
random_noise = np.log10(np.random.normal(loc=0.0, scale=0.05, size=len(dfpoolcut_non_reads)) + 1)


plt.scatter(dfpoolcut_non_reads.loc[:, 'kaiju_norm'] + random_noise, [1] * len(dfpoolcut_non_reads), label='Kaiju', alpha=0.8)
plt.plot([np.mean(dfpoolcut_non_reads.loc[:, 'kaiju_norm'])] * 2, [1 - 0.25, 1 + 0.25], alpha=0.8, c='#454545')

plt.scatter(dfpoolcut_non_reads.loc[:, 'kraken_2_norm'] + random_noise, [2] * len(dfpoolcut_non_reads), label='Kraken2', alpha=0.8)
plt.plot([np.mean(dfpoolcut_non_reads.loc[:, 'kraken_2_norm'])] * 2, [2 - 0.25, 2 + 0.25], alpha=0.8, c='#454545')

plt.scatter(dfpoolcut_non_reads.loc[:, 'krakenuniq_norm'] + random_noise, [3] * len(dfpoolcut_non_reads), label='KrakenUniq', alpha=0.8)
plt.plot([np.mean(dfpoolcut_non_reads.loc[:, 'krakenuniq_norm'])] * 2, [3 - 0.25, 3 + 0.25], alpha=0.8, c='#454545')

plt.scatter(dfpoolcut_non_reads.loc[:, 'centrifuge_norm'] + random_noise, [4] * len(dfpoolcut_non_reads), label='Centrifuge', alpha=0.8)
plt.plot([np.mean(dfpoolcut_non_reads.loc[:, 'centrifuge_norm'])] * 2, [4 - 0.25, 4 + 0.25], alpha=0.8, c='#454545')

plt.scatter(dfpoolcut_non_reads.loc[:, 'mean_norm'] + random_noise, [5] * len(dfpoolcut_non_reads), label='Mean')
plt.plot([np.mean(dfpoolcut_non_reads.loc[:, 'mean_norm'])] * 2, [5 - 0.25, 5 + 0.25], alpha=0.8, c='#454545')

plt.xticks([0, 1, 2, 3, 4], [0, 10, 100, 1000, 10000])
plt.yticks([])

plt.xlabel('Number of assigned reads')

## Export all tables (raw and cut) to an excel samplesheet


In [ ]:
raw_xlsx = pd.ExcelWriter(f"../results_diversity/{today}/stats_raw.xlsx", mode='a') 
cut_xlsx = pd.ExcelWriter(f"../results_diversity/{today}/stats_cut.xlsx", mode='a') 


dfpool, dfpoolcut = create_POOL_table('ARTIFICIAL', profilers, dict_FASTQ_len, cutoff_NA=1, cutoff_CV=1.0, cutoff_min_reads=100)
dfpool.to_excel(raw_xlsx, sheet_name='ARTIFICIAL', index=False)
dfpoolcut.to_excel(cut_xlsx, sheet_name='ARTIFICIAL', index=False)



raw_xlsx.close()
cut_xlsx.close()

Regarding the non-assigned reads, it looks like kaiju and kraken2 (kraken2 more than kaiju) has lower spurious asignment rates; whereas krakenuniq and centrifuge have high assignment rates. However, this assignment rates are, on average, 1-2 orders of magnitude lower than the number of reads of the other species in general.